There's a strange pyarrow dependency workaround that makes this only usable on CUDA gpus with the extra index pip install below

In [ ]:
!pip install scikit-learn tensorflow pandas datasets peft evaluate accelerate>=0.20.1 --extra-index-url=https://pypi.nvidia.com cudf-cu11


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.


In [ ]:
#Restarts kernel without restarting runtime (I think)

# condacolab KERNEL RESTART \
# print("Restarting of kernel...") \
# get_ipython().kernel.do_shutdown(True)

In [ ]:
# Import necessary libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.utils import compute_class_weight
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
import tensorflow as tf
import pandas as pd
import numpy as np


Skip Past this to get to Fine-Tuning Section

In [ ]:
## Mount drive where training files are

##from google.colab import drive
##drive.mount('/content/drive/')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ES_train.csv')

In [ ]:
df

,Unnamed: 0,text,label,PET
0,0,Llamó la esposa hecha una furia porque lo esta...,1,Daños colaterales
1,1,"Pero, a diferencia del medicamento que saltó a...",1,Daños colaterales
2,2,los bombardeos sobre Afganistán. Victoria Clar...,1,Daños colaterales
3,3,no existe allá la posibilidad del voto directo...,1,Daños colaterales
4,4,Acaso las limitaciones señaladas —entre las cu...,1,Daños colaterales
...,...,...,...,...
1856,1856,Tuve entonces un sueño rarísimo. Soñé que esta...,0,Partes
1857,1857,Lo que todo el mundo esperaba era un fichaje d...,0,Partes
1858,1858,"Poetas árabes de lengua francesa, y algunas de...",0,Partes
1859,1859,Pedro: ¿Para qué estoy en esto? No tiene senti...,0,Partes


In [ ]:


# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2)

# # Initialize the feature extractor
# vectorizer = CountVectorizer()

# # Fit the feature extractor to the training data
# X_train_counts = vectorizer.fit_transform(X_train)

# #weights = compute_class_weight('balanced', np.unique(y), y)


# # Fit the model to the training data
# clf = MLPClassifier(solver='sgd', alpha=1e-5, learning_rate='adaptive', hidden_layer_sizes=(100,), random_state=42)
# clf.fit(X_train_counts, y_train)



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=1e-05, learning_rate='adaptive', random_state=42,
              solver='sgd')

In [ ]:
# Evaluate the model on the testing data
# y_pred = clf.predict(vectorizer.transform(X_test))
# score = f1_score(y_test, y_pred, average='macro')
# classification_report_mlp = classification_report(y_test, y_pred)

# print('Macro F1 score:', score)
# print('\nClassification Report:\n', classification_report_mlp)


Macro F1 score: 0.4931597835984788

Classification Report:
               precision    recall  f1-score   support

           0       0.52      0.11      0.18       122
           1       0.70      0.96      0.81       269

    accuracy                           0.69       391
   macro avg       0.61      0.53      0.49       391
weighted avg       0.65      0.69      0.61       391



In [ ]:
# for index, row in df.iterrows():
#     new_text = ' '.join([str(word) for word in row['text']])
#     preprocessed_text = vectorizer.transform([new_text])[0]
#     predicted_label = clf.predict(preprocessed_text)
#     df.at[index, 'predicted_label'] = predicted_label

In [ ]:
#df

In [ ]:
#df['predicted_label'].unique()

In [ ]:
#### Method actually used in paper description is below

Trying with Fine-Tuning Instead (Ensure that it's a new runtime to not have naming errors)

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [ ]:
## Mount drive where training files are

##from google.colab import drive
##drive.mount('/content/drive/')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ES_train.csv', usecols=['label', 'text'])

In [ ]:
df

,text,label
0,Llamó la esposa hecha una furia porque lo esta...,1
1,"Pero, a diferencia del medicamento que saltó a...",1
2,los bombardeos sobre Afganistán. Victoria Clar...,1
3,no existe allá la posibilidad del voto directo...,1
4,Acaso las limitaciones señaladas —entre las cu...,1
...,...,...
1856,Tuve entonces un sueño rarísimo. Soñé que esta...,0
1857,Lo que todo el mundo esperaba era un fichaje d...,0
1858,"Poetas árabes de lengua francesa, y algunas de...",0
1859,Pedro: ¿Para qué estoy en esto? No tiene senti...,0


In [ ]:
df['label'].value_counts()

1    1143
0     718
Name: label, dtype: int64

In [ ]:
##Converting Dataframe to Dataset for HF models

import datasets
from datasets import Dataset

ds = Dataset.from_pandas(df)

## To split up dataset, shuffled by default
ds = ds.train_test_split(test_size=0.2)

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1488
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 373
    })
})

In [ ]:

## You can swap out the base model here
model_checkpoint = 'distilbert/distilbert-base-multilingual-cased'

# define label maps
id2label = {0: "Non-Euphemistic", 1: "Euphemistic"}
label2id = {"Non-Euphemistic":0, "Euphemistic":1}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
## The linear parameters after attention dropout below are the ones that are trainable
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [ ]:
# tokenize training and validation datasets
tokenized_dataset = ds.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/1488 [00:00<?, ? examples/s]

Map:   0%|          | 0/373 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1488
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 373
    })
})

In [ ]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:

# define an evaluation function to pass into trainer later

def compute_metrics(p):

    f1_metric = evaluate.load("glue", "mrpc")
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")

    predictions, labels = p
    predictions = np.argmax(predictions, axis=-1)

    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    f1_metric = f1_metric.compute(predictions=predictions, references=labels)

    return {"precision": precision, "recall": recall,  "F1 and Accuracy": f1_metric}



In [ ]:
# define list of examples
# text_list = ["He was fired yesterday", "She passed gas", "I was between jobs"]

# print("Untrained model predictions:")
# print("----------------------------")
# for text in text_list:
#     # tokenize text
#     inputs = tokenizer.encode(text, return_tensors="pt")
#     # compute logits
#     logits = model(inputs).logits
#     # convert logits to label
#     predictions = torch.argmax(logits)

#     print(text + " - " + id2label[predictions.tolist()])

In [ ]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                         target_modules = ['q_lin','k_lin','v_lin','out_lin','lin1','lin2'])

In [ ]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='distilbert/distilbert-base-multilingual-cased', revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'lin1', 'k_lin', 'v_lin', 'lin2', 'q_lin', 'out_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 923,906 || all params: 136,250,116 || trainable%: 0.6780955694745977


In [ ]:
# hyperparameters
lr = 1e-5
batch_size = 4
num_epochs = 10

In [ ]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1 and accuracy
1,No log,0.653344,0.632708,1.000000,"{'accuracy': 0.6327077747989276, 'f1': 0.7750410509031198}"
2,0.670000,0.643230,0.632708,1.000000,"{'accuracy': 0.6327077747989276, 'f1': 0.7750410509031198}"
3,0.654800,0.619720,0.634146,0.991525,"{'accuracy': 0.6327077747989276, 'f1': 0.7735537190082644}"
4,0.654800,0.602030,0.667820,0.817797,"{'accuracy': 0.6273458445040214, 'f1': 0.7352380952380952}"
5,0.621100,0.596893,0.696000,0.737288,"{'accuracy': 0.6300268096514745, 'f1': 0.7160493827160493}"
6,0.592900,0.595438,0.698039,0.754237,"{'accuracy': 0.6380697050938338, 'f1': 0.7250509164969451}"
7,0.588700,0.594004,0.741228,0.716102,"{'accuracy': 0.6621983914209115, 'f1': 0.728448275862069}"
8,0.588700,0.593714,0.742616,0.745763,"{'accuracy': 0.675603217158177, 'f1': 0.7441860465116279}"
9,0.580900,0.593268,0.724696,0.758475,"{'accuracy': 0.6648793565683646, 'f1': 0.7412008281573499}"
10,0.572600,0.593049,0.732510,0.754237,"{'accuracy': 0.6702412868632708, 'f1': 0.74321503131524}"


Trainer is attempting to log a value of "{'accuracy': 0.6327077747989276, 'f1': 0.7750410509031198}" of type <class 'dict'> for key "eval/F1 and Accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.6327077747989276, 'f1': 0.7750410509031198}" of type <class 'dict'> for key "eval/F1 and Accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.6327077747989276, 'f1': 0.7735537190082644}" of type <class 'dict'> for key "eval/F1 and Accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.6273458445040214, 'f1': 0.7352380952380952}" of type <class 'dict'> for key "eval/F1 and Accuracy" as a scalar. This invocation of Tensorboard's writer.

TrainOutput(global_step=3720, training_loss=0.6090206617950111, metrics={'train_runtime': 304.0847, 'train_samples_per_second': 48.934, 'train_steps_per_second': 12.233, 'total_flos': 691588341182112.0, 'train_loss': 0.6090206617950111, 'epoch': 10.0})

In [ ]:
#Cuda works for colab
model.to('cuda')

print("Trained model predictions:")
print("--------------------------")


testing_df = pd.read_csv('/content/drive/MyDrive/ES_test2.csv')


for idx, row in testing_df.iterrows():
    # Get the text from the row
    text = row["text"]

    inputs = tokenizer.encode(text, return_tensors="pt").to("cuda")

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(f"{idx} -- {id2label[predictions.tolist()[0]]}")





Trained model predictions:
--------------------------
0 -- Non-Euphemistic
1 -- Euphemistic
2 -- Euphemistic
3 -- Euphemistic
4 -- Euphemistic
5 -- Euphemistic
6 -- Euphemistic
7 -- Euphemistic
8 -- Non-Euphemistic
9 -- Euphemistic
10 -- Euphemistic
11 -- Euphemistic
12 -- Euphemistic
13 -- Euphemistic
14 -- Non-Euphemistic
15 -- Euphemistic
16 -- Euphemistic
17 -- Euphemistic
18 -- Euphemistic
19 -- Euphemistic
20 -- Euphemistic
21 -- Euphemistic
22 -- Euphemistic
23 -- Euphemistic
24 -- Euphemistic
25 -- Euphemistic
26 -- Non-Euphemistic
27 -- Euphemistic
28 -- Euphemistic
29 -- Euphemistic
30 -- Euphemistic
31 -- Euphemistic
32 -- Euphemistic
33 -- Non-Euphemistic
34 -- Euphemistic
35 -- Non-Euphemistic
36 -- Euphemistic
37 -- Euphemistic
38 -- Euphemistic
39 -- Non-Euphemistic
40 -- Non-Euphemistic
41 -- Euphemistic
42 -- Euphemistic
43 -- Non-Euphemistic
44 -- Euphemistic
45 -- Euphemistic
46 -- Euphemistic
47 -- Euphemistic
48 -- Euphemistic
49 -- Non-Euphemistic
50 -- Non-Euphem

In [ ]:
#for i in range(len)

In [ ]:
## This just saves the model without config and accompanying files, thus complicating inference
## without any merging afterwards

trainer.save_model("./es_euph_distil_3.0")

In [ ]:
#from huggingface_hub import notebook_login

In [ ]:
#notebook_login()

In [ ]:
### Pushes the classification epochs/training info to hugging face
## May need name editing afterwards

trainer.push_to_hub("HUGGINGFACE_USERNAME/NAME_OF_CLASSIFIER")

In [ ]:
## Gets peft data
peft_model_id = 'HUGGINGFACE_USERNAME/NAME_OF_LORA'

In [ ]:
## Merges fine-tuning with base model to get config
model_to_merge = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
# Does the merging for inference
merged_model = model_to_merge.merge_and_unload()


In [ ]:
## Saves final model for reproducibility
merged_model.save_pretrained("HUGGINGFACE_USERNAME/NAME_OF_FINAL_MERGED_MODEL")

In [ ]:
#from huggingface_hub import notebook_login

In [ ]:
#notebook_login()

In [ ]:
## Pushes final model to huggingface
merged_model.push_to_hub("HUGGINGFACE_USERNAME/NAME_OF_FINAL_MERGED_MODEL")

In [ ]:
## Tokenizer needs to be pushed too with the final merged model
tokenizer.push_to_hub(repo_id="HUGGINGFACE_USERNAME/NAME_OF_FINAL_MERGED_MODEL")